In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

24/04/15 22:26:14 WARN Utils: Your hostname, Jaders-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.58 instead (on interface en0)
24/04/15 22:26:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/15 22:26:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/15 22:26:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = spark.read.format("csv")\
    .option("header","true")\
    .option("mode","FAILFAST")\
    .option("inferSchema","true")\
    .load("apache-spark-with-data-bricks-for-data-engineering-main/data/apple_data/apple_products.csv")

In [8]:
df.printSchema()

root
 |-- Product Name: string (nullable = true)
 |-- Product URL: string (nullable = true)
 |-- Brand: string (nullable = true)
 |-- Sale Price: integer (nullable = true)
 |-- Mrp: integer (nullable = true)
 |-- Discount Percentage: integer (nullable = true)
 |-- Number Of Ratings: integer (nullable = true)
 |-- Number Of Reviews: integer (nullable = true)
 |-- Upc: string (nullable = true)
 |-- Star Rating: double (nullable = true)
 |-- Ram: string (nullable = true)



In [15]:
df.count()

62

In [33]:
df.select("Mrp").show(5)

+-----+
|  Mrp|
+-----+
|49900|
|84900|
|84900|
|77000|
|77000|
+-----+
only showing top 5 rows



In [37]:
from pyspark.sql.functions import max, col

df.select(max(col("Mrp"))).show(10)



+--------+
|max(Mrp)|
+--------+
|  149900|
+--------+



In [38]:
from pyspark.sql.functions import min, col

df.select(min(col("Mrp"))).show()

+--------+
|min(Mrp)|
+--------+
|   39900|
+--------+



In [39]:
from pyspark.sql.functions import *

In [40]:
df.where("Mrp = 149900").show()

+--------------------+--------------------+-----+----------+------+-------------------+-----------------+-----------------+----------------+-----------+----+
|        Product Name|         Product URL|Brand|Sale Price|   Mrp|Discount Percentage|Number Of Ratings|Number Of Reviews|             Upc|Star Rating| Ram|
+--------------------+--------------------+-----+----------+------+-------------------+-----------------+-----------------+----------------+-----------+----+
|APPLE iPhone 12 P...|https://www.flipk...|Apple|    140900|149900|                  6|              542|               42|MOBFWBYZ5UY6ZBVA|        4.5|4 GB|
|APPLE iPhone 12 P...|https://www.flipk...|Apple|    140900|149900|                  6|              545|               42|MOBFWBYZTHSXKMGW|        4.5|4 GB|
+--------------------+--------------------+-----+----------+------+-------------------+-----------------+-----------------+----------------+-----------+----+



In [41]:
df.where("Mrp = 39900").show()

+--------------------+--------------------+-----+----------+-----+-------------------+-----------------+-----------------+----------------+-----------+----+
|        Product Name|         Product URL|Brand|Sale Price|  Mrp|Discount Percentage|Number Of Ratings|Number Of Reviews|             Upc|Star Rating| Ram|
+--------------------+--------------------+-----+----------+-----+-------------------+-----------------+-----------------+----------------+-----------+----+
|APPLE iPhone SE (...|https://www.flipk...|Apple|     29999|39900|                 24|            95807|             8154|MOBFWQ6BGWDVGF3E|        4.5|2 GB|
|APPLE iPhone SE (...|https://www.flipk...|Apple|     29999|39900|                 24|            95909|             8161|MOBFWQ6BR3MK7AUG|        4.5|4 GB|
+--------------------+--------------------+-----+----------+-----+-------------------+-----------------+-----------------+----------------+-----------+----+



In [43]:
df.where("Mrp > 50000").show(5)

+--------------------+--------------------+-----+----------+-----+-------------------+-----------------+-----------------+----------------+-----------+----+
|        Product Name|         Product URL|Brand|Sale Price|  Mrp|Discount Percentage|Number Of Ratings|Number Of Reviews|             Upc|Star Rating| Ram|
+--------------------+--------------------+-----+----------+-----+-------------------+-----------------+-----------------+----------------+-----------+----+
|APPLE iPhone 8 Pl...|https://www.flipk...|Apple|     84900|84900|                  0|             3431|              356|MOBEXRGVAC6TJT4F|        4.6|2 GB|
|APPLE iPhone 8 Pl...|https://www.flipk...|Apple|     84900|84900|                  0|             3431|              356|MOBEXRGVGETABXWZ|        4.6|2 GB|
|APPLE iPhone 8 (S...|https://www.flipk...|Apple|     77000|77000|                  0|            11202|              794|MOBEXRGVMZWUHCBA|        4.5|2 GB|
|APPLE iPhone 8 (G...|https://www.flipk...|Apple|     7700

In [44]:
df.createOrReplaceTempView("apple_table")

In [47]:
spark.sql("SELECT DISTINCT `Product Name` FROM apple_table").show(5)

+--------------------+
|        Product Name|
+--------------------+
|APPLE iPhone 8 (S...|
|APPLE iPhone 11 (...|
|APPLE iPhone 12 P...|
|APPLE iPhone 11 (...|
|APPLE iPhone 12 (...|
+--------------------+
only showing top 5 rows



In [51]:
df.withColumn("dis_price", col("Mrp") * 0.1).select("Product Name","Mrp","dis_price")\
.withColumn("new_price", col("Mrp")-col("dis_price"))\
.orderBy(col("new_price").desc()).show(10)

+--------------------+------+---------+---------+
|        Product Name|   Mrp|dis_price|new_price|
+--------------------+------+---------+---------+
|APPLE iPhone 12 P...|149900|  14990.0| 134910.0|
|APPLE iPhone 12 P...|149900|  14990.0| 134910.0|
|APPLE iPhone 11 P...|140300|  14030.0| 126270.0|
|APPLE iPhone 11 P...|140300|  14030.0| 126270.0|
|APPLE iPhone 12 P...|139900|  13990.0| 125910.0|
|APPLE iPhone 12 P...|139900|  13990.0| 125910.0|
|APPLE iPhone 12 P...|139900|  13990.0| 125910.0|
|APPLE iPhone 12 P...|139900|  13990.0| 125910.0|
|APPLE iPhone 11 P...|131900|  13190.0| 118710.0|
|APPLE iPhone 11 P...|131900|  13190.0| 118710.0|
+--------------------+------+---------+---------+
only showing top 10 rows



In [52]:
df.write.mode("overwrite").partitionBy("Product Name").save("apple_output.csv")